In [1]:
import os
os.chdir('../py')

import gc

import pandas as pd
import numpy as np
import feather

from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams['font.family'] = 'Ricty Diminised'
sns.set(style="whitegrid", font='Ricty Diminised', palette="muted", color_codes=True, rc={'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

import japanize_matplotlib

from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import LabelEncoder

# my modules
import EDA
import utils

from scipy.stats import ks_2samp

In [6]:
import super_aggre

In [58]:
train = feather.read_dataframe('../input/feather/train.ftr')
test = feather.read_dataframe('../input/feather/test.ftr')

In [59]:
# =============================================================================
# load feature
# =============================================================================
FEATURE_DIR = '../feature/'
USE_FEATURE = ['f202']
if len(USE_FEATURE) > 0:
    tr_files = []
    te_files = []
    for f in USE_FEATURE:
        tr_file   = glob(f'../feature/{f}*tr*.ftr')
        tr_files += tr_file

        te_file   = glob(f'../feature/{f}*te*.ftr')
        te_files += te_file
else:
    tr_feature_path = '../feature/*tr*.ftr'
    te_feature_path = '../feature/*te*.ftr'

    tr_files = sorted(glob(tr_feature_path))
    te_files = sorted(glob(te_feature_path))

train = pd.concat([train, *[feather.read_dataframe(f) for f in tr_files]], axis=1)
test  = pd.concat([test, *[feather.read_dataframe(f) for f in te_files]], axis=1)

In [60]:
use_cols = ['TransactionAmt', 'dist1', 'dist2', 'f202__date_block', 'card1']

drop_cols = [col for  col in test.columns if col not in use_cols]

train.drop(columns=(drop_cols + ['isFraud']), inplace=True)

test.drop(columns=drop_cols, inplace=True)

In [61]:
df = pd.concat([train, test], axis=0)

In [62]:
df['f202__date_block'].unique()

array([201712, 201801, 201802, 201803, 201804, 201805, 201806, 201807,
       201808, 201809, 201810, 201811, 201812])

In [65]:
df['date_block_num'] = df['f202__date_block'].map(dict(zip(df['f202__date_block'].unique(), range(df['f202__date_block'].nunique()))))

In [66]:
agg = df.groupby('date_block_num', as_index=False).agg({'TransactionAmt': ['min', 'mean', 'max', 'std']})

In [67]:
card_count = pd.crosstab(index=df['card1'],
                         columns=df['date_block_num'])

In [68]:
df[df['date_block_num'] == 1]['card1']

134339     9485
134340     4461
134341     3821
134342     8335
134343     7919
134344     3821
134345     2772
134346     1974
134347     3780
134348    12577
134349     6019
134350     2803
134351    18249
134352     1675
134353     4461
134354    14290
134355     9026
134356    12544
134357    12932
134358     7676
134359     7921
134360    12260
134361     3821
134362     2801
134363     7919
134364     7861
134365     9006
134366     7919
134367    10568
134368    12725
          ...  
226819     6019
226820     1974
226821     6752
226822     6019
226823     5606
226824     6019
226825    10859
226826     3786
226827    18080
226828     7815
226829     3154
226830     2616
226831    14037
226832     4100
226833    15484
226834     9026
226835    16659
226836    11157
226837    15885
226838    17480
226839     5663
226840     1129
226841    11151
226842     7919
226843     2616
226844    15651
226845     7012
226846     8320
226847    11218
226848    16659
Name: card1, Length: 925

In [69]:
df['date_block_num'].max()

12

In [70]:
np.arange(1, 13)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [71]:
range(1, 12)

range(1, 12)

In [78]:
tmp = feather.read_dataframe('../feature/f103__test__ce_and_day_feature.ftr').columns

In [80]:
[col for col in tmp if 'full' in col]

['f103__card1_count_full', 'id_34_count_full', 'id_36_count_full']

In [82]:
df['date_block_num'].value_counts().sort_index()

0     134339
1      92510
2      85725
3     101968
4      83571
5      89673
6       2754
7      75731
8      76252
9      72692
10     80127
11     81707
12    120182
Name: date_block_num, dtype: int64

In [72]:
lags = [0, 1, 2, 3, 6]

for lag in lags:
    df[f'card1_counts__lag{lag}'] = np.nan
    
    for num in range(lag, 13):
        df.loc[df['date_block_num'] == num, f'card1_counts__lag{lag}'] = df[df['date_block_num'] == num]['card1'].map(card_count[num - lag])

In [28]:
num

11

In [73]:
df

,TransactionAmt,card1,dist1,dist2,f202__date_block,card1_counts__lag0,date_block_num,card1_counts__lag1,card1_counts__lag2,card1_counts__lag3,card1_counts__lag6
0,68.500000,13926,19.0,NaN,201712,7.0,0,NaN,NaN,NaN,NaN
1,29.000000,2755,NaN,NaN,201712,167.0,0,NaN,NaN,NaN,NaN
2,59.000000,4663,287.0,NaN,201712,178.0,0,NaN,NaN,NaN,NaN
3,50.000000,18132,NaN,NaN,201712,813.0,0,NaN,NaN,NaN,NaN
4,50.000000,4497,NaN,NaN,201712,2.0,0,NaN,NaN,NaN,NaN
5,49.000000,5937,36.0,NaN,201712,3.0,0,NaN,NaN,NaN,NaN
6,159.000000,12308,0.0,NaN,201712,39.0,0,NaN,NaN,NaN,NaN
7,422.500000,12695,NaN,NaN,201712,1407.0,0,NaN,NaN,NaN,NaN
8,15.000000,2803,NaN,NaN,201712,1243.0,0,NaN,NaN,NaN,NaN
9,117.000000,17399,19.0,NaN,201712,332.0,0,NaN,NaN,NaN,NaN


In [17]:
df['card1_count_lag1'] = np.zeros(len(df))

In [19]:
df.loc[df['date_block_num'] == 1, 'card1_count_lag1'] = df[df['date_block_num'] == 1]['card1'].map(card_count[0])

In [29]:
df.loc[df['date_block_num'] == 2, 'card1_count_lag1'] = df[df['date_block_num'] == 2]['card1'].map(card_count[1])

In [30]:
df.loc[df['date_block_num'] == 12, 'card1_count_lag1'] = df[df['date_block_num'] == 12]['card1'].map(card_count[11])

In [15]:
df[df['date_block_num'] == 1]['card1'].map(card_count[0])

134339     123
134340     759
134341     464
134342      10
134343    2698
134344     464
134345     410
134346     319
134347       3
134348     508
134349    2777
134350    1243
134351     181
134352     345
134353     759
134354     322
134355     421
134356    1340
134357     198
134358     460
134359      12
134360      94
134361     464
134362     127
134363    2698
134364     585
134365      10
134366    2698
134367     203
134368      45
          ... 
226819    2777
226820     319
226821       0
226822    2777
226823     109
226824    2777
226825      48
226826      39
226827      22
226828     397
226829    1143
226830    1245
226831      78
226832      12
226833     252
226834     421
226835    1347
226836     341
226837    2633
226838     235
226839      12
226840      16
226841       6
226842    2698
226843    1245
226844     286
226845       2
226846     397
226847     158
226848    1347
Name: card1, Length: 92510, dtype: int64

In [12]:
card_count

date_block_num,0,1,2,3,4,5,6,7,8,9,10,11,12
card1,,,,,,,,,,,,,
1000,0,0,1,0,0,0,0,0,0,0,0,0,0
1001,1,2,0,0,0,0,0,0,0,0,0,0,2
1002,0,0,0,0,0,0,0,0,0,0,0,0,1
1003,0,0,0,0,0,0,0,0,0,0,0,1,0
1004,2,0,0,1,0,2,0,0,0,0,0,1,2
1005,1,0,0,0,0,0,0,0,0,0,0,0,1
1006,1,0,1,0,1,0,0,1,0,0,0,0,5
1007,1,0,1,0,0,0,0,0,0,0,0,2,0
1008,0,0,2,0,0,1,0,1,0,0,0,1,1


In [34]:
len(test)

506691

In [41]:
len(train) + len(test)

1097231

In [38]:
df.iloc[:len(train)]

,TransactionAmt,card1,dist1,dist2,f202__date_block,date_block_num,card1_count_lag1,card1_counts__lag1,card1_counts__lag2,card1_counts__lag3,card1_counts__lag6
0,68.500000,13926,19.0,NaN,201712,0,0.0,NaN,NaN,NaN,NaN
1,29.000000,2755,NaN,NaN,201712,0,0.0,NaN,NaN,NaN,NaN
2,59.000000,4663,287.0,NaN,201712,0,0.0,NaN,NaN,NaN,NaN
3,50.000000,18132,NaN,NaN,201712,0,0.0,NaN,NaN,NaN,NaN
4,50.000000,4497,NaN,NaN,201712,0,0.0,NaN,NaN,NaN,NaN
5,49.000000,5937,36.0,NaN,201712,0,0.0,NaN,NaN,NaN,NaN
6,159.000000,12308,0.0,NaN,201712,0,0.0,NaN,NaN,NaN,NaN
7,422.500000,12695,NaN,NaN,201712,0,0.0,NaN,NaN,NaN,NaN
8,15.000000,2803,NaN,NaN,201712,0,0.0,NaN,NaN,NaN,NaN
9,117.000000,17399,19.0,NaN,201712,0,0.0,NaN,NaN,NaN,NaN


In [39]:
df.iloc[len(train):]

,TransactionAmt,card1,dist1,dist2,f202__date_block,date_block_num,card1_count_lag1,card1_counts__lag1,card1_counts__lag2,card1_counts__lag3,card1_counts__lag6
0,31.950001,10409,1.0,NaN,201807,7,0.0,1.0,1.0,1.0,1.0
1,49.000000,4272,4.0,NaN,201807,7,0.0,4.0,4.0,4.0,4.0
2,171.000000,4476,2635.0,NaN,201807,7,0.0,0.0,0.0,0.0,0.0
3,284.950012,10989,17.0,NaN,201807,7,0.0,6.0,6.0,6.0,6.0
4,67.949997,18018,6.0,NaN,201807,7,0.0,11.0,11.0,11.0,11.0
5,57.950001,12839,NaN,NaN,201807,7,0.0,16.0,16.0,16.0,16.0
6,87.000000,16560,NaN,NaN,201807,7,0.0,16.0,16.0,16.0,16.0
7,390.000000,15066,303.0,NaN,201807,7,0.0,46.0,46.0,46.0,46.0
8,103.949997,2803,3.0,NaN,201807,7,0.0,33.0,33.0,33.0,33.0
9,117.000000,12544,8.0,NaN,201807,7,0.0,48.0,48.0,48.0,48.0


In [42]:
feather.read_dataframe('../feature/f203__test__numeric_lag.ftr')

,f203__index,f203__date_block_num,f203__TransactionAmt_min__lag1,f203__TransactionAmt_mean__lag1,f203__TransactionAmt_max__lag1,f203__TransactionAmt_std__lag1,f203__dist1_min__lag1,f203__dist1_mean__lag1,f203__dist1_max__lag1,f203__dist1_std__lag1,...,f203__TransactionAmt_max__lag6,f203__TransactionAmt_std__lag6,f203__dist1_min__lag6,f203__dist1_mean__lag6,f203__dist1_max__lag6,f203__dist1_std__lag6,f203__dist2_min__lag6,f203__dist2_mean__lag6,f203__dist2_max__lag6,f203__dist2_std__lag6
0,590540,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,...,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946
1,590541,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,...,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946
2,590542,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,...,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946
3,590543,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,...,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946
4,590544,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,...,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946
5,590545,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,...,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946
6,590546,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,...,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946
7,590547,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,...,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946
8,590548,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,...,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946
9,590549,7,1.001,171.459625,3163.939941,316.236938,0.0,116.683731,2751.0,351.194336,...,5278.950195,228.731796,0.0,134.430771,6818.0,405.398346,0.0,258.679382,6915.0,559.944946


In [40]:
len(feather.read_dataframe('../feature/f203__test__numeric_lag.ftr'))

590540

In [36]:
len(feather.read_dataframe('../feature/f203__train__numeric_lag.ftr'))

590540